# Query OpenVINO Device and Model Properties

This notebook uses the [Query Device API](https://docs.openvino.ai/latest/openvino_docs_OV_UG_query_api.html) to check and display the Device Properties for all available compute devices in the system. It also downloads the person-detection-0002 model from Open Model Zoo, compiles it on each device with various performance hints, and displays the resulting Compiled Model properties.

## Imports

In [ ]:
from openvino.runtime import Core

## Check device properties

In [ ]:
# Check device properties
ie = Core()
for device in ie.available_devices:
    print('%s\tSUPPORTED_PROPERTIES:\n' % device)
    for property_key in ie.get_property(device, 'SUPPORTED_PROPERTIES'):
        if property_key not in ('SUPPORTED_METRICS', 'SUPPORTED_CONFIG_KEYS', 'SUPPORTED_PROPERTIES'):
            try:
                property_val = ie.get_property(device, property_key)
            except TypeError:
                property_val = 'UNSUPPORTED TYPE'
            print('\t%s: %s' % (property_key, property_val))
    print('\n')

## Download test model

We use a pre-trained person detection model from OpenVINO's [Open Model Zoo](https://docs.openvino.ai/nightly/model_zoo.html) to compile on the devices.

In [ ]:
# directory where model will be downloaded
base_model_dir = "model"

# model name as named in Open Model Zoo
model_name = "person-detection-0202"
precision = "FP16"
model_path = (
    f"model/intel/{model_name}/{precision}/{model_name}.xml"
)
download_command = f"omz_downloader " \
                   f"--name {model_name} " \
                   f"--precision {precision} " \
                   f"--output_dir {base_model_dir} " \
                   f"--cache_dir {base_model_dir}"
! $download_command

### Compile the model

In [ ]:
# Read model
model = ie.read_model(model=model_path)

# Compile model and check model properties on each device
devices = ie.available_devices + ['AUTO']

for device_name in devices:
    hints = ['LATENCY','THROUGHPUT']
    if device_name == 'AUTO':
        hints = hints + ['CUMULATIVE_THROUGHPUT']
    for hint in hints:
        # Compile model on device with performance hint
        compiled_model = ie.compile_model(model, device_name, config={'PERFORMANCE_HINT':'%s' % hint})
        
        # Print compiled model propreties
        print('DEVICE: %s\tPERFORMANCE HINT: %s\n' % (device_name,hint))
        property_keys = compiled_model.get_property('SUPPORTED_PROPERTIES')

        for key in property_keys:
            if key != 'SUPPORTED_PROPERTIES':
                prop = compiled_model.get_property(key)
                print('\t%s: %s' % (key, prop))
            
        print('\n')
        del compiled_model